In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### Data wrangling

In [2]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [5]:
#remove rows with missing entries in queries, responses or labels
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


### Text preprocessing

In [7]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words


#remove single character tokens

def remove_single_characters(text):
    filtered_words = [word for word in text if len(word) > 1]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

# Lemmatize with POS Tag
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemmatized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text]
    return lemmatized

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
            
        if 'single_characters' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_single_characters(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        if 'lemmatize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lemmatize(x))    
            
        return self.df_column

In [9]:
unprocessed_df = df.copy()

In [10]:
#preprocess columns 'question1' and 'question2'
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','single_characters','stemming']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [11]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [13]:
#create dataframe of top 100 queries that have label = 1
queries = df[df['is_duplicate'] == 1][:100]
queries = queries[['qid1','question1','qid2','question2', 'is_duplicate']]
queries = queries.reset_index(drop=True)
queries.head()

,qid1,question1,qid2,question2,is_duplicate
0,536040,"[control, horni, emot]",536041.0,"[control, horni]",1.0
1,155721,"[one, mbb]",7256.0,[mbb],1.0
2,147570,"[best, self, help, book, read, chang, life]",787.0,"[top, self, help, book, read]",1.0
3,71243,"[hillari, clinton, polici, toward, india, beco...",177376.0,"[hilari, clinton, polici, toward, india, becom...",1.0
4,22332,"[best, book, studi, tensor, gener, rel, basic]",22333.0,"[best, book, tensor, calculu]",1.0


In [16]:
#utility class for printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Sentence matching with TFIDF -  sklearn implementation

In [17]:
corpus = df[['qid2','question2']]

corpus.drop_duplicates(subset='qid2', inplace=True)

#sklearn tfidf - l1 normalization method
from sklearn.feature_extraction.text import TfidfVectorizer
train_questions = list(corpus['question2'])
query_questions = list(queries['question1'])


tf1 = TfidfVectorizer(tokenizer=None,preprocessor=None, analyzer = (lambda x:x),lowercase=False, norm='l1')
X_train = tf1.fit_transform(train_questions)
X_test = tf1.transform(query_questions)

matched = np.array((X_test@X_train.T).todense())
matched_sorted = np.zeros(matched.shape, dtype='int')

sktop2_l1 = []
sktop5_l1 = []

for i in range(matched.shape[0]):
    matched_sorted[i,:] = matched[i,:].argsort()[::-1]
    top2ind = matched_sorted[i,:2]
    top5ind = matched_sorted[i,:5]
    top2docs = [corpus.iloc[c,0] for c in top2ind]
    top5docs = [corpus.iloc[c,0] for c in top5ind]
    if queries.iloc[i,2] in top2docs:
        sktop2_l1.append(1)
    else:
        sktop2_l1.append(0)
    if queries.iloc[i,2] in top5docs:
        sktop5_l1.append(1)
    else:
        sktop5_l1.append(0)

tf2 = TfidfVectorizer(tokenizer=None,preprocessor=None, analyzer = (lambda x:x),lowercase=False, norm='l2')
X_train2 = tf2.fit_transform(train_questions)
X_test2 = tf2.transform(query_questions)
matched2 = np.array((X_test2@X_train2.T).todense())
matched_sorted2 = np.zeros(matched2.shape, dtype='int')

sktop2_l2 = []
sktop5_l2 = []

for i in range(matched2.shape[0]):
    matched_sorted2[i,:] = matched2[i,:].argsort()[::-1]
    top2ind = matched_sorted2[i,:2]
    top5ind = matched_sorted2[i,:5]
    top2docs = [corpus.iloc[c,0] for c in top2ind]
    top5docs = [corpus.iloc[c,0] for c in top5ind]
    if queries.iloc[i,2] in top2docs:
        sktop2_l2.append(1)
    else:
        sktop2_l2.append(0)
    if queries.iloc[i,2] in top5docs:
        sktop5_l2.append(1)
    else:
        sktop5_l2.append(0)

from sklearn.metrics import accuracy_score
sktop2_l1_acc = accuracy_score(queries['is_duplicate'], sktop2_l1)
sktop5_l1_acc = accuracy_score(queries['is_duplicate'], sktop5_l1)
sktop2_l2_acc = accuracy_score(queries['is_duplicate'], sktop2_l2)
sktop5_l2_acc = accuracy_score(queries['is_duplicate'], sktop5_l2)
print(color.BOLD+'Sklearn implementation'+color.END)
print()
print('Accuracy of sentence matching with TFIDF "l1" normalization method...')
print()
print('Retrieving correct response in top 2 matched responses: {:.1f} %'.format(sktop2_l1_acc*100))
print('Retrieving correct response in top 5 matched responses: {:.1f} %'.format(sktop5_l1_acc*100))
print()
print()
print('Accuracy of sentence matching with TFIDF "l2" normalization method...')
print()
print('Retrieving correct response in top 2 matched responses: {:.1f} %'.format(sktop2_l2_acc*100))
print('Retrieving correct response in top 5 matched responses: {:.1f} %'.format(sktop5_l2_acc*100))
      

<ipython-input-17-bfd566458b64>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus.drop_duplicates(subset='qid2', inplace=True)


Sklearn implementation

Accuracy of sentence matching with TFIDF "l1" normalization method...

Retrieving correct response in top 2 matched responses: 16.0 %
Retrieving correct response in top 5 matched responses: 32.0 %


Accuracy of sentence matching with TFIDF "l2" normalization method...

Retrieving correct response in top 2 matched responses: 45.0 %
Retrieving correct response in top 5 matched responses: 61.0 %


### Sentence Matching using TF-IDF method - implementation from scratch

In [18]:
#create a vocabulary list from tokens in responses
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
len(q2vocab)

50010

In [19]:
# create dictionary - word in vocabulary: list of response ids (q2id) it is found in
Word_Doc_ids = {}
for word in q2vocab:
    Word_Doc_ids[word] = []
for i in range(len(df)):
    for word in df.loc[i,'question2']:
        Word_Doc_ids[word].append(df.loc[i,'qid2'])

#create set to drop duplicate values of doc_ids
for word in q2vocab:
    Word_Doc_ids[word] = list(set(Word_Doc_ids[word]))
    
    
list(Word_Doc_ids.items())[3:5]

[('nemo',
  [303233.0,
   513861.0,
   496678.0,
   413610.0,
   172205.0,
   246991.0,
   413781.0,
   456023.0]),
 ('bitdefend', [114823.0, 71175.0, 11719.0, 26538.0, 26539.0])]

In [20]:
#create Inverse Document_frequency (IDF) dictionary - word in vocabulary : IDF
#use idf formula as per sklearn implementation - for smoothing add 1 to numerator and denominator

N = df['qid2'].nunique()
print('Number of unique responses: {}'.format(N))

IDF = {}
for word in Word_Doc_ids.keys():
    IDF[word] = np.log((1+N/(len(Word_Doc_ids[word]) +1))+1)
list(IDF.items())[:5]    

Number of unique responses: 273121


[('ski', 9.116692476478114),
 ('fprove', 11.824537664272047),
 ('chinapakisthan', 11.824537664272047),
 ('nemo', 10.320511524763331),
 ('bitdefend', 10.725954665792823)]

In [21]:
#dictionary of list of words in each response - q2id : words
doc_words = dict(zip(df['qid2'],df['question2']))

list(doc_words.items())[:2]

[(213222.0, ['level', 'preprat', 'enough', 'exam', 'jlpt']),
 (536041.0, ['control', 'horni'])]

In [22]:
#define function to calculate  norm - to apply 'l1' or 'l2' normalization for calculation of term frequency

def norm(list_of_words, normalization_method = 'l1'):
    word_vec = np.array([list_of_words.count(word) for word in set(list_of_words)])
    if normalization_method == 'l1':
        norm = np.linalg.norm(word_vec,ord=1)
    else:
        norm = np.linalg.norm(word_vec, ord=2)
    return norm


In [23]:
# Create a dictionary where keys are words in the vocabulary, 
#values are dictionary of key:values as q2id : term frequency for corresponding word
#nested dictionary - word: {{q2id1: tf}, {q2id2: tf}, ....}
#use 'l1' norm to normalize term frequency

inverted_tf_l1 = {}
for word in q2vocab:
    word_tf = {}
    for doc in Word_Doc_ids[word]:
        tf = doc_words[doc].count(word)/norm(doc_words[doc],'l1')
        word_tf[doc] = tf
    inverted_tf_l1[word] = word_tf   

In [24]:
#use 'l2' norm to normalize term frequency

inverted_tf_l2 = {}
for word in q2vocab:
    word_tf = {}
    for doc in Word_Doc_ids[word]:
        tf = doc_words[doc].count(word)/norm(doc_words[doc],'l2')
        word_tf[doc] = tf
    inverted_tf_l2[word] = word_tf 

#### Creating inverted file for retreiving tfidf values - term frequencies normalized by either 'l1' or 'l2' norm

In [25]:
# Create a dictionary where keys are words in the vocab, 
#values are dictinary of key:values as doc_id:tfidf for corresponding word
#nested dictionary - word: {{doc_id1: tf}, {doc_id2: tf}, ....}

inverted_tfidf_l1 = {}
for word in q2vocab:
    doc_tfidfs = []
    for tup in list(inverted_tf_l1[word].items()):
        tup = list(tup)
        tfidf = tup[1]*IDF[word]
        tup[1] = tfidf
        doc_tfidfs.append(tup)
    inverted_tfidf_l1[word] = doc_tfidfs  
    
    
inverted_tfidf_l2 = {}
for word in q2vocab:
    doc_tfidfs = []
    for tup in list(inverted_tf_l2[word].items()):
        tup = list(tup)
        tfidf = tup[1]*IDF[word]
        tup[1] = tfidf
        doc_tfidfs.append(tup)
    inverted_tfidf_l2[word] = doc_tfidfs     

In [26]:
for i in range(len(inverted_tfidf_l1)):
    inverted_tfidf_l1[q2vocab[i]] = {x[0]:x[1] for x in inverted_tfidf_l1[q2vocab[i]]}
    
for i in range(len(inverted_tfidf_l2)):
    inverted_tfidf_l2[q2vocab[i]] = {x[0]:x[1] for x in inverted_tfidf_l2[q2vocab[i]]}    

In [27]:
list(inverted_tfidf_l1.items())[3:5]

[('nemo',
  {303233.0: 1.2900639405954164,
   513861.0: 1.7200852541272218,
   496678.0: 2.580127881190833,
   413610.0: 2.580127881190833,
   172205.0: 2.0641023049526663,
   246991.0: 1.1467235027514813,
   413781.0: 1.4743587892519043,
   456023.0: 3.4401705082544436}),
 ('bitdefend',
  {114823.0: 2.1451909331585646,
   71175.0: 2.681488666448206,
   11719.0: 2.681488666448206,
   26538.0: 3.0645584759408067,
   26539.0: 1.7876591109654705})]

In [28]:
#define function to access unprocessed question from original dataframe
def retreive_unprocessed_question(question_qid, question_type):
    """
    Parameters:
        question_qid : 'qid1' or 'qid2' 
        
        question_type : 'question1' or 'question2'
        
   Returns: 
       unprocessed question : question1 or question2 from original unprocessed dataframe
    """
    if question_type == 'query':
        unprocessed_question = unprocessed_df.loc[unprocessed_df['qid1']==question_qid, 'question1'].iloc[0]
    else:
        unprocessed_question = unprocessed_df.loc[unprocessed_df['qid2']==question_qid, 'question2'].iloc[0]
    
    return unprocessed_question


#define function to retrieve matching responses against one query
from collections import Counter
def get_tfidfRanked_responses(normalization_method,query_index,number_of_responses, print_results = False ):
    
    """
    Parameters:
        normalization_method : 'l1' or 'l2'
    
        query_index : 0 to 99 - first 100 queries that have label =  1

        number_of_responses : top2 or top5 numbers of matching responses against a query
        
        print_results : If true, prints the unprocessed query, ground truth response and top2 or top5 matching responses obtained by sentence embedding method
    
    Returns:
        matched_responses : list of 'qid2' for ranked number of matching responses
    """
    
    query = queries.loc[query_index,'question1']
    query_qid = queries.loc[query_index,'qid1']
    actual_response_id = queries.loc[query_index,'qid2']
    
    if normalization_method == 'l1':
        inverted_file = inverted_tfidf_l1
    else:
        inverted_file = inverted_tfidf_l2
    
    counter_objects = []
    for word in query:
        if word in inverted_file.keys():
            counter_objects.append(Counter(inverted_file[word]))
            combined = sum(counter_objects,Counter())
            matched_responses = sorted(combined, key = combined.get, reverse=True)[:number_of_responses]
    
    if print_results:
        print(color.BOLD+'Sentence matching with TF-IDF method - implementation from scratch...'+color.END) 
        print()
    
        print(color.BOLD+'Query'+color.END)
        print(color.RED+retreive_unprocessed_question(query_qid,'query')+color.END)
        print()
        print(color.BOLD+'Ground truth (actual) matching response'+color.END)
        print(color.CYAN +retreive_unprocessed_question(actual_response_id,'response')+color.END)
        print()
        print(color.BOLD+'Top {} matched responses by TFIDF method, {} normalization'.format(number_of_responses,normalization_method)+color.END)
        for response_id in matched_responses:
            if response_id == actual_response_id:
                print(color.CYAN+retreive_unprocessed_question(response_id,'response')+color.END)
            else:
                print(retreive_unprocessed_question(response_id,'response'))
    else:        
        
        return matched_responses 
    
    
# define function to get matching responses against all queries
def all_queries_responses(normalization_method,number_of_responses):
    response_labels = []
    for i in range(len(queries)):
        matched_responses = get_tfidfRanked_responses(normalization_method,i,number_of_responses, print_results = False )
        if queries.loc[i,'qid2'] in matched_responses:
            response_labels.append(1.0)
        else:
            response_labels.append(0.0)
            
    return response_labels
   
        


In [29]:
#get prediction labels for matched responses using 'l1' or 'l2' normalization methods
top2_pred_l1 = all_queries_responses('l1',2)
top5_pred_l1 = all_queries_responses('l1',5)
top2_pred_l2 = all_queries_responses('l2',2)
top5_pred_l2 = all_queries_responses('l2',5)

In [30]:
top2_l1 = accuracy_score(queries['is_duplicate'],top2_pred_l1)
top5_l1 = accuracy_score(queries['is_duplicate'],top5_pred_l1)
top2_l2 = accuracy_score(queries['is_duplicate'],top2_pred_l2)
top5_l2 = accuracy_score(queries['is_duplicate'],top5_pred_l2)

print(color.BOLD+"Implementation from scratch"+color.END)
print()
print('Accuracy of sentence matching with TFIDF "l1" normalization method...')
print()
print('Retrieving correct response in top 2 matched responses: {:.1f} %'.format(top2_l1*100))
print('Retrieving correct response in top 5 matched responses: {:.1f} %'.format(top5_l1*100))
print()
print()
print('Accuracy of sentence matching with TFIDF "l2" normalization method...')
print()
print('Retrieving correct response in top 2 matched responses: {:.1f} %'.format(top2_l2*100))
print('Retrieving correct response in top 5 matched responses: {:.1f} %'.format(top5_l2*100))
      

Implementation from scratch

Accuracy of sentence matching with TFIDF "l1" normalization method...

Retrieving correct response in top 2 matched responses: 16.0 %
Retrieving correct response in top 5 matched responses: 30.0 %


Accuracy of sentence matching with TFIDF "l2" normalization method...

Retrieving correct response in top 2 matched responses: 47.0 %
Retrieving correct response in top 5 matched responses: 58.0 %


### View matching responses for selected queries

In [31]:
get_tfidfRanked_responses(normalization_method='l1', query_index=1, number_of_responses=5, print_results = True)

Sentence matching with TF-IDF method - implementation from scratch...

Query
What can one do after MBBS?

Ground truth (actual) matching response
What do i do after my MBBS ?

Top 5 matched responses by TFIDF method, l1 normalization
What have to Do after MBBS?
What can I do after my MBBs?
What do i do after my MBBS ?
What should one do and should not do in 40s?
How can I do mbbs after doing BDS?


In [32]:
get_tfidfRanked_responses(normalization_method='l2', query_index=1, number_of_responses=5, print_results = True)

Sentence matching with TF-IDF method - implementation from scratch...

Query
What can one do after MBBS?

Ground truth (actual) matching response
What do i do after my MBBS ?

Top 5 matched responses by TFIDF method, l2 normalization
What have to Do after MBBS?
What can I do after my MBBs?
What do i do after my MBBS ?
How can I do mbbs after doing BDS?
How should I study in MBBS?


In [33]:
get_tfidfRanked_responses(normalization_method='l1', query_index=15, number_of_responses=2, print_results = True)

Sentence matching with TF-IDF method - implementation from scratch...

Query
What are some of the most beautiful houses in the world?

Ground truth (actual) matching response
Which are some of the most beautiful houses around the world?

Top 2 matched responses by TFIDF method, l1 normalization
What is the beauty?
Why am I beautiful?


In [34]:
get_tfidfRanked_responses(normalization_method='l2', query_index=15, number_of_responses=2, print_results = True)

Sentence matching with TF-IDF method - implementation from scratch...

Query
What are some of the most beautiful houses in the world?

Ground truth (actual) matching response
Which are some of the most beautiful houses around the world?

Top 2 matched responses by TFIDF method, l2 normalization
What are some of the most beautiful houses in the world?
Which are some of the most beautiful houses around the world?
